# Deep Learning for NLP with PyTorch

저자: [Robert Guthrie](//github.com/rguthrie3/DeepLearningForNLPInPytorch)  
역자: [Don Kim](//github.com/dgkim5360)

## Sequence Models and Long-Short Term Memory Networks
원본: http://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

지금까지 우리는 여러 feed-forward network들을 봐 왔는데, feed-forward network는 상태(state)를 계속 간직할 필요가 없었다. 이것은 어쩌면 우리가 원하지 않는 모델의 성질일 수도 있다.

NLP의 핵심적인 개념으로 sequence 모델을 들 수 있는데, 이는 시간 (혹은 순서)에 따라 input 간의 의존 관계가 있다는 개념이다. Sequence 모델의 전통적인 예로, 품사 태깅을 위한 hidden Markov 모델이나 conditional random field를 들 수 있다.

Recurrent neural network은 이러한 상태 개념을 가지고 다니는 network이다. 예를 들어 network의 output이 다음 input에 포함되어 진행되면서, 예전 정보가 연속적으로 network에 계속 전파될 수 있는 것이다.

LSTM의 경우, sequence의 각 ($t$번째) 요소마다 _hidden state_ $h_t$를 갖게 되는데, 이 $h_t$는 원칙적으로 과거의 어떤 정보라도 가지고 있을 수 있다. 이 $h_t$를 이용해서 수 많은 일들을 할 수 있다. language model을 구축해서 단어를 예측할 수도 있고, 품사 태깅을 할 수도 있게 된다.

### LSTM's in Pytorch
본격적인 예제에 들어가기 전에, 몇 가지 알아둘 것이 있다.

Pytorch의 LSTM은 모든 input을 3D tensor라고 가정하고 받는데, 이 세 가지 축이 의미하는 바를 알아두는 것이 중요하다.
1. 첫 번째 축은 sequence 순서 정보를 의미한다.
2. 두 번째 축은 mini-batch의 순서 정보를 의미하는데, 여기서는 mini-batch에 대해 다룬 적이 없으므로 무조건 1차원을 가진다고 가정하겠다.
3. 세 번째 축은 input 순서를 갖는다.

만약 sequence 모델에 "The cow jumped"라는 문장을 넣고 돌린다면, input은 아래와 같이 생길 것이다.

\begin{equation}
\begin{bmatrix}
\overbrace{q_{\text{The}}}^{\text{row vector}} \\
q_{\text{cow}} \\
q_{\text{jumped}}
\end{bmatrix}
\end{equation}

이 와중에 위 tensor의 1차원이라 나타나지 않는 두 번째 축이 있음을 기억하자.

또한, sequence를 차례차례 보내지 않고 한 번에 보낼 수도 있다. 이 경우에는 첫 번째 축 역시 1차원이 될 것이다.

아래 간단한 예제를 보자.

In [1]:
# Author: Robert Guthrie
# Translator: Don Kim
import torch

torch.manual_seed(1)

In [2]:
lstm = torch.nn.LSTM(3, 3)  # Input은 3차원, output도 3차원
inputs = [torch.autograd.Variable(torch.randn(1, 3))
          for _ in range(5)]  # sequence의 길이는 5

# Hidden state를 초기화한다.
hidden = (torch.autograd.Variable(torch.randn(1, 1, 3)),
          torch.autograd.Variable(torch.randn((1, 1, 3))))
for i in inputs:
    # sequence의 요소 하나씩 단계적으로 진행한다.
    # 매 단계마다 hidden은 hidden state 정보를 담게 된다.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

In [3]:
# Sequence 전체를 한 번에 진행해도 똑같은 결과를 얻을 수 있다.
# LSTM이 주는 output 중 첫 번째 값인 "out"은 sequence 모두를 진행하면서 얻게 되는
# 모든 hidden state를 담고 있다.
# 두 번째 output인 "hidden"은 가장 최근의 hidden state만을 갖고 있다.
# ("out"의 마지막과 "hidden"의 값이 같은 것을 확인해보자)
# LSTM이 이렇게 두 가지 output을 주는 이유를 말해보자면,
# "out"을 통해 sequence 전체의 hidden state에 접근할 수 있는 것에 가치가 있고,
# "hidden"은 LSTM의 argument로 쓰여서 sequence의 backpropagate를 계속
# 진행할 수 있게 해주는 것이다.

# LSTM에 넣기 위해 두 번째 축을 추가해서 3D tensor로 만들자.
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
# Hidden state를 초기화한다.
hidden = (torch.autograd.Variable(torch.randn(1, 1, 3)),
          torch.autograd.Variable(torch.randn((1, 1, 3))))
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

Variable containing:
(0 ,.,.) = 
 -0.0187  0.1713 -0.2944

(1 ,.,.) = 
 -0.3521  0.1026 -0.2971

(2 ,.,.) = 
 -0.3191  0.0781 -0.1957

(3 ,.,.) = 
 -0.1634  0.0941 -0.1637

(4 ,.,.) = 
 -0.3368  0.0959 -0.0538
[torch.FloatTensor of size 5x1x3]

(Variable containing:
(0 ,.,.) = 
 -0.3368  0.0959 -0.0538
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
 -0.9825  0.4715 -0.0633
[torch.FloatTensor of size 1x1x3]
)


### Example: An LSTM for Part-of-Speech Tagging
이번에는 LSTM을 이용해서 품사 태깅을 해보려고 한다. 또한 여기서는 [Viterbi](https://en.wikipedia.org/wiki/Viterbi_algorithm)나 [Forward-Backward](https://en.wikipedia.org/wiki/Forward%E2%80%93backward_algorithm)와 같은 알고리즘은 사용하지 않을 것이지만, LSTM이 어떻게 돌아가는 지 경험한 후에는 언급한 알고리즘들이 어떻게 사용될 수 있을지 생각해보는 것도 독자들에게 좋은 (꽤 어려운) 연습 문제가 될 것 같다.

품사 태깅 모델을 설명하겠다.
* Input 문장을 $w_1, \cdots, w_M$ 이라고 한다.  
  $w_i \in V$, 즉  각 단어 $w_i$는 단어장 $V$ 안에 속해있다.
* $T$를 품사 모음으로 표시한다. 그리고 $y_i$를 단어 $w_i$의 품사로 표시한다.
* 이제 단어 $w_i$의 품사에 대한 우리의 예측을 $\hat{y}_i$로 표시한다.

이 모델은 [structured prediction](https://en.wikipedia.org/wiki/Structured_prediction)을 하게 된다. 이는 모델의 output이 sequence 형태의 $\hat{y}_1, \cdots, \hat{y}_M$ $(\hat{y}_i \in T)$인 것을 의미한다.

이제 예측을 하기 위해서 LSTM 모델에 문장을 던져줘야 한다. $i$ 번째 순서의 hidden state를 $h_i$라고 표시하겠다. 그리고 모든 품사 태그에 고유한 숫자를 부여하겠다. 이는 지난 번 word embedding 예제에서 `word_to_ix`를 만들었던 것과 똑같은 이치이다. 그러면 $\hat{y}_i$를 계산하기 위한 규칙은 다음과 같다.

\begin{equation}
\hat{y}_i = \arg\max_j \left( \log\text{Softmax}(Ah_i + b) \right)_j
\end{equation}

~~Hidden state를 affine map에 태운 것을 log softmax로 취한 후에, 그 결과 중 가장 큰 값을 갖는 품사 태그로 예측한다는 의미이다.~~ Affine map $A$의 target space 차원 값이 $|T|$인 점도 알아두자.

실제 코드로 들어가보겠다. Data를 준비하자.

In [4]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return torch.autograd.Variable(tensor)

In [5]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"]),
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# 아래 값들은 보통 32 또는 64 차원으로 사용하지만
# 여기서는 우리가 train하는 weight들이 어떻게 변하는지 직접 볼 수 있도록
# 작게 설정한다.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


모델을 만들겠다.

In [6]:
class LSTMTagger(torch.nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = torch.nn.Embedding(vocab_size,
                                                  embedding_dim)
        
        # LSTM은 word embedding과 hidden 차원값을 input으로 받고,
        # hidden state를 output으로 내보낸다.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)
        
        # Hidden state space에서 tag space로 보내는 linear layer를 준비한다.
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # Hidden state는 자동적으로 만들어지지 않으므로 직접 기능을 만들겠다.
        # 3D tensor의 차원은 각각 (layer 개수, mini-batch 개수, hidden 차원)
        # 을 의미한다. 왜 이렇게 해야만 하는지 궁금하다면 Pytorch 문서를 참고 바란다.
        return (
            torch.autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
            torch.autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
        )
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1),
            self.hidden
        )
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = torch.nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores

이제 모델을 훈련시키겠다.

In [7]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM,
                   len(word_to_ix), len(tag_to_ix))
loss_function = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Training하기 전의 품사 태깅 점수를 보겠다.
# Output의 (i, j) 원소는 i번째 단어가 j번째 품사일 점수를 나타낸다.
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
print(tag_scores)

# 재차 말하지만, 보통 300번의 epoch를 돌리지 않는다.
# 이건 매우 단순한 예제니까 가능한 거다.
for epoch in range(300):
    for sentence, tags in training_data:
        # Step 1. Torch에서 gradient는 축적된다는 기억하자.
        # 새로운 데이터를 넣기 전에, 기존 gradient 정보를 날려줘야 한다.
        model.zero_grad()
        
        # 또한 LSTM의 이전 단계 hidden state와 분리시키면서
        # hidden state를 초기화해줘야 한다.
        model.hidden = model.init_hidden()
        
        # Step 2. Network에 넣을 수 있도록 input 자료를 알맞게 변환해준다.
        # 즉, 단어 인덱스에 맞게 Variable로 변환해준다.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        # Step 3. Forward pass를 돌려라.
        tag_scores = model(sentence_in)
        
        # Step 4. Loss, gradient를 계산하고,
        # optimizer.step()을 통해 parameter를 업데이트한다.
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# Training이 끝난 후의 결과를 보겠다.
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
# "the dog ate the apple"이라는 문장을 넣었다.
# 출력물의 (i,j)번째 원소는 i번째 단어가 j번째 품사일 점수이다.
# 그 중 최고 점수를 기록한 품사를 선택하게 된다.
# 여기서 5개 순서마다 최고 점수 인덱스는 0 1 2 0 1이고, 이에 연결되는 품사는
# DET NOUN VERB DET NOUN (관사 명사 동사 관사 명사)이다. 정답!
print(tag_scores)

Variable containing:
-1.1989 -0.9630 -1.1497
-1.2522 -0.9158 -1.1586
-1.2563 -1.0022 -1.0550
-1.1518 -1.1443 -1.0065
-1.1728 -1.0677 -1.0593
[torch.FloatTensor of size 5x3]

Variable containing:
-0.1902 -1.8654 -3.9957
-4.1051 -0.0263 -4.6590
-4.0204 -3.1797 -0.0614
-0.0372 -4.3504 -3.7448
-4.0387 -0.0348 -4.1001
[torch.FloatTensor of size 5x3]



### Exercise: Augmenting the LSTM Part-of-Speech Tagger with Character-Level Features
지금까지 다룬 LSTM 예제에서 각 단어는 embedding을 갖게 되고, 이 embedding이 sequence 모델의 input으로 들어갔었다. 이제 이 word embedding에 더해서 단어의 각 철자들에서 생기는 효과를 얹어보려고 한다. _-ly_ 로 끝나는 영어 단어는 부사일 가능성이 굉장히 높다는 점을 생각해보면, 철자 레벨의 정보가 우리에게 굉장한 도움을 줄 수 있겠다 생각할 수 있다.

이제 몇 가지 표기와 규칙을 정하겠다.
* $c_w$를 단어 $w$의 철자 레벨의 표현이라고 하겠다.
* $x_w$는 기존 word embedding을 의미한다.
* 그러면 sequence 모델의 input으로는 $x_w$와 $c_w$를 합친 것을 넣어줘야 한다.  
  따라서 $x_w$이 5차원이고 $c_w$가 3차원이라면, LSTM은 8차원의 input을 받아야 할 것이다.

철자 레벨의 표현을 얻기 위해서는 LSTM이 단어의 철자들을 받아서 작동시키고, 그 결과인 최종 hidden state를 $c_w$로 하면 된다. 몇 가지 힌트를 주겠다.

* 새로운 모델은 두 개의 LSTM를 갖게 될 것이다.  
  POS 태그 점수를 위한 기존 LSTM에 더해서, 각 단어의 철자 레벨의 표현을 얻기 위한 LSTM 하나가 더 추가된다.
* 철자들을 sequence 모델로 돌리기 위해서는 character embedding이 필요할 것이다. 이 character embedding이 철자를 위한 LSTM에 input으로 들어가게 될 것이다.

아직 성공하지 못했지만, 현재까지 발버둥친 역자의 흔적을 아래 남겨둔다.

In [ ]:
def prepare_sequence_char(word):
    # ASCII indices
    idxs = [ord(c) for c in word]
    tensor = torch.LongTensor(idxs)
    return torch.autograd.Variable(tensor)

In [ ]:
class LSTMTaggerEx(torch.nn.Module):
    def __init__(self,
                 char_embedding_dim, char_hidden_dim, char_size,
                 word_embedding_dim, word_hidden_dim, vocab_size,
                 tagset_size):
        super(LSTMTaggerEx, self).__init__()
        self.char_hidden_dim = char_hidden_dim
        self.word_hidden_dim = word_hidden_dim
        self.hidden_dim = char_hidden_dim + word_hidden_dim
        
        self.char_embeddings = torch.nn.Embedding(char_size,
                                                  char_embedding_dim)
        self.word_embeddings = torch.nn.Embedding(vocab_size,
                                                  word_embedding_dim)
        
        self.char_lstm = torch.nn.LSTM(char_embedding_dim, char_hidden_dim)
        # Word embedding과 character represenation의 차원을 합해야 한다.
        self.lstm = torch.nn.LSTM(word_embedding_dim + char_hidden_dim,
                                  self.word_hidden_dim)
        
        self.hidden2tag = torch.nn.Linear(word_hidden_dim, tagset_size)
        self.char_hidden = self.init_char_hidden()
        self.word_hidden = self.init_word_hidden()
        
    def init_word_hidden(self):
        return (
            torch.autograd.Variable(torch.zeros(1, 1, self.word_hidden_dim)),
            torch.autograd.Variable(torch.zeros(1, 1, self.word_hidden_dim)),
        )
    
    def init_char_hidden(self):
        return (
            torch.autograd.Variable(torch.zeros(1, 1, self.char_hidden_dim)),
            torch.autograd.Variable(torch.zeros(1, 1, self.char_hidden_dim)),
        )
    
    def forward(self, sentence, sentence_char):
        for word in sentence_char:
            char_embeds = self.char_embeddings(sentence_char)
            char_lstm_out, self.char_hidden = self.char_lstm(
                char_embeds.view(len(sentence_char), 1, -1),
                self.char_hidden
            )
        word_embeds = self.word_embeddings(sentence)
        embeds = torch.cat([word_embeds, self.char_hidden])
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1),
            self.hidden
        )
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = torch.nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
ASCII_SIZE = 256
model_ex = LSTMTaggerEx(EMBEDDING_DIM, HIDDEN_DIM, ASCII_SIZE,
                        EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix),
                        len(tag_to_ix))
loss_function_ex = torch.nn.NLLLoss()
optimizer_ex = torch.optim.SGD(model_ex.parameters(), lr=0.1)

for epoch in range(300):
    for sentence, tags in training_data:
        # Step 1. 기존 gradient 정보 초기화
        model_ex.zero_grad()
        
        # 또한 LSTM의 이전 단계 hidden state와 분리시키면서 hidden state를 초기화
        model_ex.char_hidden = model_ex.init_char_hidden()
        model_ex.word_hidden = model_ex.init_word_hidden()
        
        # Step 2. Network에 넣을 수 있도록 input 자료를 알맞게 변환해준다.
        # 즉, 단어 인덱스에 맞게 Variable로 변환해준다.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        sentence_char = prepare_sequence_char(sentence)
        targets = prepare_sequence(tags, tag_to_ix)
        
        # Step 3. Forward pass
        tag_scores = model_ex(sentence_in, sentence_char)
        
        # Step 4. Loss, gradient를 계산, parameter 업데이트
        loss = loss_function_ex(tag_scores, targets)
        loss.backward()
        optimizer_ex.step()

# Training 결과
inputs = prepare_sequence(training_data[0][0], word_to_ix)
inputs_char = prepare_sequence_char(training_data[0][0])
tag_scores = model(inputs, inputs_char)
print(tag_scores)